Here, I create a neural network for the MNIST dataset; because, do you really work on AI if you don't have an MNIST implementation of a neural network?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim

##import our mnist
from torch.utils.data import TensorDataset
from torch.autograd import Variable
import torchvision
from torchvision import transforms
from torchvision.datasets import MNIST

SEED = 1


let's see if we have CUDA. I'm using a free colab notebook with a GPU.

In [ ]:
cuda = torch.cuda.is_available()
cuda

# Loading the data

let's load the MNIST dataset. we do it like this:




In [ ]:
train = MNIST('./data', train=True, download=True, transform=transforms.Compose([
  transforms.ToTensor() #ToTensor for the MinMax normalization
]))
test = MNIST('./data', train=False, download=True, transform=transforms.Compose([
      transforms.ToTensor() #ToTensor for the MinMax normalization
]))

we gon' form the dataloader for training and testing, and then we gon' make the image tensor of all 600000 images in the MNIST traning dataset









In [ ]:
dataloader_kwargs = dict(shuffle=True, batch_size=256, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64) 

train_loader = dataloader.DataLoader(train, **dataloader_kwargs)
test_loader = dataloader.DataLoader(test, **dataloader_kwargs)

train_data = train.train_data

let's look at the train data tensor

In [ ]:
train_data.shape

print('[Train]')
print(' - Numpy Shape:', train.train_data.cpu().numpy().shape)
print(' - Tensor Shape:', train.train_data.size())
print(' - min:', torch.min(train_data))
print(' - max:', torch.max(train_data))

let's go and view an image with our image viewer function:

In [ ]:
def show_images(batch):
  im = torchvision.utils.make_grid(batch)
  plt.figure(figsize=(10, 10))
  plt.imshow(np.transpose(im.numpy(), (1,2,0))) #np.transpose takes in 2 arguments the input array (or image), and the transpose axis as a tuple
  plt.show()

dataiter = iter(train_loader)
images, labels = dataiter.next()
my_batch = torch.narrow(images, 0, 128, 32)
batch_labels = torch.narrow(labels,0, 128, 32)
#print("images.shape")
#print(images.shape)

#print("my_batch shape")
#print(my_batch.shape)

show_images(my_batch)


here are the labels too:

In [ ]:
print(batch_labels.numpy())

# Defining our model, our train, and test functions

so here's the model I created. it's a convolutional net I made in my AI course at UIUC, and I want to test it on the MNIST.

In [ ]:
class convNet(torch.nn.Module):
    def __init__(self, lrate, in_size, out_size, momentum):
      super(convNet, self).__init__()

      #you need the layers, the loss function, and the optimizer

      self.conv1 = nn.Conv2d(1, 10, 5)
      self.hidden1 = nn.Linear(10*12*12, 300) #put the pooled features through a hidden layer
      self.output = nn.Linear(300, out_size) #this layer classifies for us

      self.relu = nn.ReLU()
      self.pool = nn.MaxPool2d(2,2)


      self.optimizer = optim.SGD(self.parameters(), lr=lrate, momentum=momentum)
      self.loss_fn = nn.CrossEntropyLoss



    def get_parameters(self):
        return self.parameters()

    def forward(self, x):
        #implement forward propogation
        x = x.view(-1, 1, 28, 28)
        x = self.pool(self.relu(self.conv1(x)))
        x = x.view(-1, 10*12*12)
        x = self.relu(self.hidden1(x))
        x = self.output(x)
        return x

    def step(self, x, y):
        #perform a gradient descent step

        #ALWAYS zero the gradient before you step
        self.optimizer.zero_grad()

        outputs = self.forward(x)
        predicted = torch.max(outputs.data, 1)

        #get the loss and backpropagate it
        loss = self.loss_fn(predicted, y)
        loss.backward()

        self.optimizer.step()
        L = loss.item()
        return L


Now that we have the model defined, we will create the train function for the model to use. 

In [ ]:
def train(model, train_loader, n_epochs=5):
  losses = []
  for t in range(n_epochs):
    curr_loss = 0.0
    for i, data_batch in enumerate(train_loader):
      data, labels = data_batch
      curr_loss += model.step(data, labels)

    losses.append(curr_loss)
      


Create the test function too for our accuracy:

In [ ]:
def test(model, test_loader, n):
  num_correct = 0
  for data_batch in test_loader:
    input, labels = data_batch
    predicted = model(input)
    num_correct += (predicted == labels).sum()
  
  return (num_correct/n)*100.0


# Fitting our function

now that we have our train, test, and model defined, let's train our model!

In [ ]:
##Our parameters to pass in the model. 'learn_rate' and 'momentum' are some hyperparamters we tune
learn_rate = 1e-3
momentum = 0.9
input_size = 784
output_size = 10

model = convNet(lrate=learn_rate, in_size=input_size, out_size=output_size, momentum=momentum) 

let's train!

In [ ]:
train(model=model, train_loader=train_loader, n_epochs=6)